Data Cleaning

In [ ]:
#import libraries/modules
import datetime as dt
import numpy as np
import pandas as pd
import re
import html
from bs4 import BeautifulSoup
from ast import literal_eval

In [ ]:
#Set dataframe visualization row and column limts
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 100)

Part 1 - Intial Cleaning

In [ ]:
#read csv as pandas df
df1 = pd.read_csv("cleaner_merged_data.csv", low_memory=False)

In [ ]:
#create new df with just ids
df2 = df1[["steam_appid"]]

In [ ]:
#create column with boolean with True for when df1 name_x is not null
df2.loc[:,"has_name"]  = df1.loc[:,"name_x"].notna()

In [ ]:
#for the age column, create a new one,  deleting the + singns at the end of some of the strings
df2.loc[:, "required_age"]  = df1.loc[:, "required_age"].str.replace("+", "")

In [ ]:
#turn strings into int
df2.loc[:, "required_age"]  = df2.loc[:, "required_age"].astype(float).astype(int)

In [ ]:
#add the is free column
df2.loc[:, "is_free"]  = df1.loc[:, "is_free"]

In [ ]:
#create column with boolean for when this column is not null
df2.loc[:,"has_about_game"]  = df1.loc[:,"about_the_game"].notna()
df2.loc[:,"has_detailed_description"]  = df1.loc[:,"detailed_description"].notna()
df2.loc[:,"has_short_description"]  = df1.loc[:,"short_description"].notna()

In [ ]:
#list of full audio text in various languages
FullAudio = ["(all with full audio support)", "languages with full audio support", "idiomas con localización de audio", "med fuld lydunderstøttelse", "ęzyki z pełnym udźwiękowieniem",
                             "- スペインフル音声対応言語", "フル音声対応言語", "озвучивание доступно на этих языках", "idiomas com suporte total de áudio", "Langues avec support audio complet", "具有完全音频支持的语言", 
                             "lingue con supporto audio completo", "Sprachen mit voller Audiounterstützung", "bahasa dengan dukungan audio penuh", "(text only)", "(full audio)"]

In [ ]:
#function to remove html tangs and text between brackets
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    clean_text = soup.get_text()
    clean_text = html.unescape(clean_text)
    clean_text = re.sub(r'<[^<]+?>', '', clean_text)
    clean_text = re.sub(r'\[.*?\]', '', clean_text)
    return clean_text

#function to further clean text by removing the full audio text and asterisks
def clean_text(text):
    if isinstance(text, str):
        clean = remove_html_tags(text)
        cleaner = clean.replace("*", "")
        for i in FullAudio:
            if i in cleaner:
                cleaner = cleaner.replace(i, "")
        text = cleaner
    else: 
        text = text 
    return text


In [ ]:
#apply clean text function to languages column
df1["supported_languages"] = df1["supported_languages"].apply(clean_text)

In [ ]:
#get all unique language values
all_languages = set()
for languages_str in df1['supported_languages']:
    if isinstance(languages_str, str):
        languages = languages_str.split(', ')
    for lang in languages:
        all_languages.add(lang.strip())

In [ ]:
#put them in list and print
languages_list = list(all_languages)
for index, language in enumerate(languages_list):
    print(index, language)

In [ ]:
#create dictionary mappinng all variations of all unique languages
languages = {}
languages["Yoruba"] = ["Yoruba"]
languages["Russian"] = ["Ruso", "Russisksprog", "Russian", "Bhs. Rusia", "ロシア語", "Russisk", "Russisch", "русский","俄语", "Rosyjski", "Russe", "Russo"]
languages["Odia"] = ["Odia"]
languages["Chinese"] = ["Chino simplificado","繁体中文","Chiński uproszczony","Simplified Chinese","Cinese tradizionale","Chiński tradycyjny","Chinois traditionnel","Chinês tradicional","Traditional Chinese","Bhs. Tionghoa Sederhana","Forenklet kinesisk","简体中文","Chinesisch (vereinfacht)","китайский (упр.)","Chino tradicional", "Chinesisch (traditionell)", "китайский (трад.)", "中国語（簡体字)", "Chinês simplificado", "Chinois simplifié", "Bhs. Tionghoa Tradisional"]
languages["Danish"] = ["Danish","Danese", "датский"]
languages["Punjabi"] = ["Punjabi (Gurmukhi)", "Punjabi (Shahmukhi)"]
languages["Romanian"] = ["Romanian", "румынский"]
languages["Polish"] = ["Polacco", "Polish", "Polonais", "波兰语", "Polnisch", "Polaco", "Polski", "Bhs. Polandia", "Polsk", "польский"]
languages["Korean"] = ["Bhs. Korea", "Koreanisch", "Korean", "корейский", "Coreano", "韩语", "韓国語", "Koreansk","Koreański", "Coréen"]
languages["English"] = ["Angielski", "Engelsk", "英語", "Inglês", "Inglese", "Bhs. Inggris", "английский", "Englisch", "Inglés", "英语", "Anglais", "English"]
languages["Marathi"] = ["Marathi"]
languages["Norwegian"] = ["Norwegian", "норвежский"]
languages["Turkish"] = ["Turco", "Bhs. Turki", "Tureckij", "Turkish", "турецкий", "Turkmen"]
languages["Albanian"]= ["Albanian"]
languages["French"] = ["Bhs. Prancis","Francese", "Fransk", "French", "Francês", "Francés", "法语", "Francuski", "フランス語", "Français","Französisch", "французский"]
languages["Czech"] = ["Czech", "Tcheco", "чешский", "捷克语", "Tschechisch", "Bhs. Ceko"]
languages["Italian"] =["Włoski", "Italienisch", "イタリア語", "Italian", "意大利语", "итальянский", "Italien", "Italiano", "Italiensk", "Bhs. Italia"]
languages["Slovak"]=["Slovak", "#lang_slovakian"]
languages["Zulu"]=["Zulu"]
languages["Kyrgyz"]=["Kyrgyz"]
languages["Hungarian"]=["Bhs. Hungaria", "венгерский", "Hungarian","Ungarisch"]
languages["Sotho"]=["Sotho"]
languages["Spanish"]=["西班牙语 - 西班牙","Spanish - Spain","Spagnolo - Spagna","Hiszpański latynoamerykański","Espagnol - Amérique latine","スペイン語","スペイン語 - スペイン", "Hiszpański latynoamerykańskij", "Espagnol - Espagne","Spagnolo - America Latina","Spanisch – Spanien","Hiszpański", "Español de España", "Espanhol (Espanha)", "Bhs. Spanyol - Amerika Latin", "Bhs. Spanyol - Spanyol", "Spansk – Spanien", "испанский Лат. Ам.", "Espanhol (América Latina)", "испанский", "Spanish - Latin America"]
languages["Macedonian"]=["Macedonian"]
languages["German"]=["Niemiecki","German", "Tysk", "Alemão", "Alemán", "Tedesco", "Bhs. Jerman", "German;", "ドイツ語", "德语", "Allemand", "немецкий", "Deutsch"]
languages["Scots"]=["Scots"]
languages["Slovenian"]=["Slovenian"]
languages["Uyghur"]=["Uyghur"]
languages["Thai"]=["Bhs. Thai", "Thai"]
languages["Indonesian"]=["Indonésio", "Indonezyjski", "Indonesian"]
languages["Arabic"]=["Bhs. Arab", "Árabe","阿拉伯语", "Arabo", "Arabic", "Arabski"]
languages["Tamil"]=["Tamil"]
languages["Japanese"]=["Japonés","Bhs. Jepang", "японский", "日语", "Japanese", "日本語", "Japansk", "Japonais", "Giapponese", "Japonês", "Japoński", "Japanisch"]
languages["Lithuanian"]=["Lithuanian"]
languages["Assamese"]=["Assamese"]
languages["Portuguese"]=["Portugisisk – Brasilien","Português (Brasil)","ポルトガル語－ブラジル","Portugalski brazylijski","Portugais du Brésil","Brasilianisches Portugiesisch","бр. португальский","Portuguese - Portugal", "Portoghese - Brasile", "Portugués de Portugal", "Portugués de Brasil", "Portuguese - Brazil", "Portugalski", "португальский", "葡萄牙语 - 巴西", "Bhs. Portugis - Brasil"]
languages["Telugu"]=["Telugu"]
languages["Finnish"]=["Finnish","финский"]
languages["Cherokee"]=["Cherokee"]
languages["Ukrainian"]=["Ukrainian", "украинский", "Bhs. Ukraina"]
languages["Swahili"]=["Swahili"]
languages["Tswana"]=["Tswana"]
languages["Kinyarwanda"]=["Kinyarwanda"]
languages["Tatar"]=["Tatar"]
languages["Mongolian"]=["Mongolian"]
languages["Sinhala"]=["Sinhala"]
languages["Hausa"]=["Hausa"]
languages["Maori"]=["Maori"]
languages["Galician"]=["Galician"]
languages["Estonian"]=["Estonian"]
languages["Bosnian"]=["Bosnian"]
languages["Tigrinya"]=["Tigrinya"]
languages["Nepali"]=["Nepali"]
languages["Catalan"]=["Catalan", "Valencian"]
languages["Hebrew"]=["Hebrew"]
languages["Kannada"]=["Kannada"]
languages["Sindhi"]=["Sindhi"]
languages["Persian"]=["Persa", "Persian", "Tajik", "Dari"]
languages["Icelandic"]=["Icelandic"]
languages["Bengali"]=["Bangla"]
languages["Quechua"]=["Quechua"]
languages["Azerbaijani"]=["Azerbaijani"]
languages["Serbian"]=["Serbian"]
languages["Uzbek"]=["Uzbek"]
languages["Dutch"]=["нидерландский", "Dutch", "Niederländisch", "荷兰语", "Olandese"]
languages["Kazakh"]=["Kazakh"]
languages["Swedish"]=["шведский", "Svensksprog", "Swedish"]
languages["Malayalam"]=["Malayalam"]
languages["Maltese"]=["Maltese"]
languages["Wolof"]=["Wolof"]
languages["Afrikaans"]=["Afrikaans"]
languages["Basque"]=["Basque"]
languages["Latvian"]=["Latvian"]
languages["Armeian"]=["Armenian"]
languages["Irish"]=["Irish"]
languages["Belarusian"]=["Belarusian"]
languages["Vietnamese"]=["Vietnamese", "Vietnamita"]
languages["Konkani"]=["Konkani"]
languages["Hindi"]=["Hindi"]
languages["Bulgarian"]=["Bulgarian", "болгарский"]
languages["Georgian"]=["Georgian"]
languages["K'iche'"]=["K'iche'"]
languages["Amharic"]=["Amharic"]
languages["Malay"]=["Malay"]
languages["Khmer"]=["Khmer"]
languages["Croatian"]=["Croatian"]
languages["Sorani"]=["Sorani"]
languages["Greek"]=["Greek", "греческий"]
languages["Welsh"]=["Welsh"]
languages["Luxembourgish"]=["Luxembourgish"]
languages["Xhosa"]=["Xhosa"]
languages["Filipino"]=["Filipino"]
languages["Igbo"]=["Igbo"]
languages["Gujarati"]=["Gujarati"]
languages["Urdu"]=["Urdu"]



In [ ]:
#create columns for each language with false as default value
for i in languages.keys():
    df2[i]= False

In [ ]:
#code for adding true to the boolean columns if the game supports that languege
for index, row in df1.iterrows():
    languages_str = row["supported_languages"]
    if isinstance(languages_str, str):
        languages_list = languages_str.split(', ')
        for i in languages_list:
            i = i.strip()
            for key, value in languages.items():
                if any(lang in languages_list for lang in value):
                    df2.loc[index, key] = True

In [ ]:
#create column with boolean with True for when this column is not null
df2.loc[:,"has_headerImage"]  = df1.loc[:,"header_image"].notna()
df2.loc[:,"has_capsuleImage"]  = df1.loc[:,"capsule_image"].notna()
df2.loc[:,"has_capsuleImagev5"]  = df1.loc[:,"capsule_imagev5"].notna()
df2.loc[:,"has_website"]  = df1.loc[:,"website"].notna()

In [ ]:
#apply clean text function to pc requirements
df1["pc_requirements"] = df1["pc_requirements"].apply(clean_text)

In [ ]:
#dictionary of pc requiremnet levels with common specs
processor = {"Low": ["i3", "ryzen 3", "athlon", "dual-core", "dual core"],
              "Medium": ["i5", "i7", "ryzen 5", "ryzen 7", "quad-core", "quad core"], 
              'High': ["i9", "ryzen 9", "threadripper", "hexa-core", "hexa core","octa-core", "octa core"]}
memory = {"Low": ["memory: 1gb", "memory: 1 gb", "memory: 1gigabytes", "memory: 1 gigabytes", "memory: 2gb", "memory: 2 gb", "memory: 2gigabytes", "memory: 2 gigabytes", "memory: 3gb", "memory: 3 gb", "memory: 3gigabytes", "memory: 3 gigabytes","memory: 4gb", "memory: 4 gb", "memory: 4gigabytes", "memory: 4 gigabytes", "memory: 5gb", "memory: 5 gb", "memory: 5gigabytes", "memory: 5 gigabytes", "memory: 6gb", "memory: 6 gb", "memory: 6gigabytes", "memory: 6 gigabytes"], 
          "Medium": ["memory: 8gb", "memory: 8 gb", "memory: 8gigabytes", "memory: 8 gigabytes", "memory: 10gb", "memory: 10 gb", "memory: 10gigabytes", "memory: 10 gigabytes", "memory: 12gb", "memory: 12 gb", "memory: 12gigabytes", "memory: 12 gigabytes", "memory: 14gb", "memory: 14 gb", "memory: 14gigabytes", "memory: 14 gigabytes"],
          "High": ["memory: 16gb", "memory: 16 gb", "memory: 16gigabytes", "memory: 16 gigabytes", "memory: 32gb", "memory: 32 gb", "memory: 32gigabytes", "memory: 32 gigabytes", "memory: 64gb", "memory: 64 gb", "memory: 64gigabytes", "memory: 64 gigabytes"]}
graphics = {"Low": ["gt 10", "gt10", "rx 4", "rx4", "intel hd", "intelhd", "directx", "shader", "graphics: 2gb", "graphics: 2 gb", "graphics: 2gigabytes", "graphics: 2 gigabytes", "graphics: 4gb", "graphics: 4 gb", "graphics: 4gigabytes", "graphics: 4 gigabytes"],
            "Medium": ["gtx 16", "gtx16", "rtx 20", "rtx20", "rx5" "rx 5", "graphics: 6gb", "graphics: 6 gb", "graphics: 6gigabytes", "graphics: 6 gigabytes", "graphics: 8gb", "graphics: 8 gb", "graphics: 8gigabytes", "graphics: 8 gigabytes"],
            "High": ["rtx30", "rtx 30", "rtx40", "rtx 40", "rx 6", "rx6", "rx7", "rx 7", "graphics: 10gb", "graphics: 10 gb", "graphics: 10gigabytes", "graphics: 10 gigabytes", "graphics: 12gb", "graphics: 12 gb", "graphics: 12gigabytes", "graphics: 12 gigabytes", "graphics: 14gb", "graphics: 14 gb", "graphics: 14gigabytes", "graphics: 14 gigabytes", "graphics: 16gb", "graphics: 16 gb", "graphics: 16gigabytes", "graphics: 16 gigabytes"]}
storage = {"Low": ["hdd"],
           "Medium": ["ssd 500", "ssd: 500", "ssd500"],
           "High": ["ssd 1t", "ssd1t"]}

In [ ]:
#new colums for separate pc reqs
df2["Processor"] = None
df2["Memory"] = None
df2["Graphics"] = None
df2["Storage"] = None

In [ ]:
#checking the row contents for the pc requirements column and updating the processor, memory, grapgics, and storage column
#based on the presence of the dictionary values 
for index, row in df1.iterrows():
    pc_reqs_str = row["pc_requirements"]
    if isinstance(pc_reqs_str, str):
        for key, value in processor.items():
            if any(req in pc_reqs_str for req in value):
                df2.loc[index, "Processor"] = key

for index, row in df1.iterrows():
    pc_reqs_str = row["pc_requirements"]
    if isinstance(pc_reqs_str, str):
        for key, value in memory.items():
            if any(req in pc_reqs_str for req in value):
                df2.loc[index, "Memory"] = key

for index, row in df1.iterrows():
    pc_reqs_str = row["pc_requirements"]
    if isinstance(pc_reqs_str, str):
        for key, value in graphics.items():
            if any(req in pc_reqs_str for req in value):
                df2.loc[index, "Graphics"] = key

for index, row in df1.iterrows():
    pc_reqs_str = row["pc_requirements"]
    if isinstance(pc_reqs_str, str):
        for key, value in storage.items():
            if any(req in pc_reqs_str for req in value):
                df2.loc[index, "Storage"] = key

In [ ]:
#apply clean text
df1["developers"] = df1["developers"].apply(clean_text)
df1["publishers"] = df1["publishers"].apply(clean_text)

In [ ]:
#create function to apply literal eval with error handling
def handle_literal_eval(x):
    try:
        return literal_eval(x)
    except (SyntaxError, ValueError):
        return x

In [ ]:
#apply literal eval to columns
df1["publishers"] = df1["publishers"].apply(lambda x: handle_literal_eval(x))
df1["developers"] = df1["developers"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#keep only first in list 
for index, row in df1.iterrows():
    devs_list = row["developers"]
    if isinstance(devs_list, list):
        df2.loc[index, "developers"] = devs_list[0]
for index, row in df1.iterrows():
    devs_list = row["publishers"]
    if isinstance(devs_list, list):
        df2.loc[index, "publishers"] = devs_list[0]

In [ ]:
#appy literal eval
df1["platforms"] = df1["platforms"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#make new colums for each platfom supported based on the key-val pairs in the original platform dictionary
for index, row in df1.iterrows():
    plat_dict = row["platforms"]
    if isinstance(plat_dict, dict):
        for key, value in plat_dict.items():
            df2.loc[index, key] = value

In [ ]:
#make boolean type
df2["windows"] = df2["windows"].astype(bool)
df2["mac"] = df2["mac"].astype(bool)
df2["linux"] = df2["linux"].astype(bool)

In [ ]:
#create column with boolean with True for when this column is not null
df2.loc[:,"has_metacritic_score"]  = df1.loc[:,"metacritic"].notna()

In [ ]:
#apply literal_eval
df1["categories"] = df1["categories"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#set of unique category ids
all_catids = set()
for cat_list in df1['categories']:
    if isinstance(cat_list, list):
        for cat_dict in cat_list:
            if isinstance(cat_dict, dict):
                cat_id = cat_dict.get("id")
                all_catids.add(cat_id)

In [ ]:
#make categories dictionary
all_categories_dict = {}
unique_names = set()
for cat_id in all_catids:
    all_categories_dict[cat_id] = []
    for cat_list in df1['categories']:
        if isinstance(cat_list, list):
            for cat_dict in cat_list:
                if isinstance(cat_dict, dict):
                    if cat_dict.get("id") == cat_id:
                        cat_name = cat_dict.get("description")
                        cat_name = cat_name.lower()
                        if cat_name not in unique_names:
                            all_categories_dict[cat_id].append(cat_name)
                            unique_names.add(cat_name)

In [ ]:
#make a new version using the english names as keys and the ids as values
new_cat_dict = {
    "multiplayer": [1],
    "singlepalyer": [2],
    "mods": [6, 19],
    "valveAntiCheat": [8],
    "coOp": [9, 38],
    "captions": [13],
    "commentary":[14],
    "stats":[15],
    "sourceSDK": [16],
    "levelEditor": [17],
    "partialControllerSupport":[18],
    "mmo": [20],
    "steamAchievements": [22],
    "steamCloud": [23],
    "sharedScreen": [24, 37, 39],
    "steamLeaderboards": [25],
    "crossPlatformMultiplayer": [27],
    "fullControlerSupport": [28],
    "steamTradingCards": [29],
    "steamWorkshop": [30, 51],
    "vrSupport": [31, 53],
    "steamTurnNotifications": [32],
    "inAppPurchases": [35],
    "steamVRCollectibles": [40],
    "remotePlay": [41, 42, 43],
    "remotePlayTogether": [44],
    "lanPvP": [47],
    "lanCoOp": [48],
    "pVp": [49],
    "trackedControllerSupport": [52],
    "vrOnly": [54],
    "hdrAvailable": [61],
    "familySharing": [62]
}

In [ ]:
#create columns for each category key 
for key in new_cat_dict.keys():
    df2[key] = False

In [ ]:
#update value for columns depending on whther the id for that category is present for that row
for index, row in df1.iterrows():
    cat_list = row["categories"]
    if isinstance(cat_list, list):
        for cat_dict in cat_list:
            if isinstance(cat_dict, dict):
                cat_id = cat_dict.get("id")
                for key, value in new_cat_dict.items():
                   if cat_id in value:
                        df2.loc[index, key] = True

In [ ]:
#apply literal eval 
df1["genres"] = df1["genres"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#set of unique ids
all_genids = set()
for gen_list in df1['genres']:
    if isinstance(gen_list, list):
        for gen_dict in gen_list:
            if isinstance(gen_dict, dict):
                gen_id = gen_dict.get("id")
                all_genids.add(gen_id)

In [ ]:
#make dictionary
all_genres_dict = {}
unique_gen_names = set()
for gen_id in all_genids:
    all_genres_dict[gen_id] = []
    for gen_list in df1['genres']:
        if isinstance(gen_list, list):
            for gen_dict in gen_list:
                if isinstance(gen_dict, dict):
                    if gen_dict.get("id") == gen_id:
                        gen_name = gen_dict.get("description")
                        gen_name = gen_name.lower()
                        if gen_name not in unique_gen_names:
                            all_genres_dict[gen_id].append(gen_name)
                            unique_gen_names.add(gen_name)


In [ ]:
#make a new version using the english names as keys and the ids as values
new_genres_dict= {
    "g_simulation": ["28"],
    "g_webPublishing": ["59"],
    "g_adventure": ["25"],
    "g_gore": ["74"],
    "g_sexual_content": ["71"],
    "g_tutorial": ["84"],
    "g_short": ["83"],
    "g_photoEditing": ["55"],
    "g_animationAndModeling": ["51"],
    "g_360video": ["85"],
    "g_gameDevelopment": ["60"],
    "g_strategy": ["2"],
    "g_indie": ["23"],
    "g_movie": ["80"],
    "g_violent": ["73"],
    "g_accounting": ["50"],
    "g_mmo": ["29"],
    "g_sports": ["18"],
    "g_education": ["54"],
    "g_utilities": ["57"],
    "g_episodic": ["82"],
    "g_videoProduction": ["58"],
    "g_rpg": ["3"],
    "g_racing": ["9"],
    "g_nudity": ["72"],
    "g_casual": ["4"],
    "g_ftp": ["37"],
    "g_action": ["1"],
    "g_audioProduction": ["52"],
    "g_softwareTraining": ["56"],
    "g_designAndIllustration": ["53"],
    "g_documentary": ["81"]
}

In [ ]:
#create columns for each category key 
for key in new_genres_dict.keys():
    df2[key] = False

In [ ]:
#update value for columns depending on whther the id for that category is present for that row
for index, row in df1.iterrows():
    gen_list = row["genres"]
    if isinstance(gen_list, list):
        for gen_dict in gen_list:
            if isinstance(gen_dict, dict):
                gen_id = gen_dict.get("id")
                for key, value in new_genres_dict.items():
                   if gen_id in value:
                        df2.loc[index, key] = True

In [ ]:
#create column with boolean with True for when this column is not null
df2.loc[:,"has_screenshots"]  = df1.loc[:,"screenshots"].notna()

In [ ]:
#create column for recommendations with 0 as default
df2["total_recommendations"] = 0

In [ ]:
#apply literal eval
df1["recommendations"] = df1["recommendations"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#get total recommendations from dictionary and add the value to the df column
for index, row in df1.iterrows():
    recommendations = row["recommendations"]
    if isinstance(recommendations, dict):
        recommendation = recommendations.get("total")
        df2.loc[index, "total_recommendations"] = recommendation

In [ ]:
#Create new columns
df2["coming_soon"] = False
df2["release_date"] = None

In [ ]:
#apply literal eval
df1["release_date"] = df1["release_date"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#get dictionary values and add them to columns
for index, row in df1.iterrows():
    rdate_dict = row["release_date"]
    if isinstance(rdate_dict, dict):
         df2.loc[index, "release_date"] = rdate_dict.get("date")
         df2.loc[index, "coming_soon"] = rdate_dict.get("coming_soon")

In [ ]:
#make datetime value and then just turn to date without timestamp
df2['release_date'] = pd.to_datetime(df2['release_date'], errors='coerce')
df2['release_date'] = df2['release_date'].dt.date

In [ ]:
#create column with boolean for when this column is not null
df2.loc[:,"has_support_info"]  = df1.loc[:,"support_info"].notna()
df2.loc[:,"has_background"]  = df1.loc[:,"background"].notna()
df2.loc[:,"has_background_raw"]  = df1.loc[:,"background_raw"].notna()

In [ ]:
#create content descriptor boolean column using the dictionary
for index, row in df1.iterrows():
    c_dict = row["content_descriptors"]
    if isinstance(c_dict, dict):
        if c_dict.get("notes") is None:
            df2.loc[index,"has_content_warning"] = False
        else:
            df2.loc[index,"has_content_warning"] = True

In [ ]:
#apply literal eval
df1["ratings"] = df1["ratings"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#function for getting average required age from the ratings dictionary
def average_required_age(row):
    sum_age = 0
    count = 0
    rating_dict = row["ratings"]  
    if isinstance(rating_dict, dict):
        for system_dict in rating_dict.values():
            if "required_age" in system_dict:
                try:
                    required_age = int(system_dict["required_age"])
                    sum_age += required_age
                    count += 1
                except ValueError:
                    pass
    if count > 0:
        average_age = sum_age / count
    else:
        average_age = 0
    
    return average_age

In [ ]:
#function for getting content descriptor from the ratings dictionary
def get_content_warning(row):
    rating_dict = row["ratings"] 
    if isinstance(rating_dict, dict):
        for system_dict in rating_dict.values():
            if "descriptors" in system_dict:
                a = system_dict.get("descriptors")
                if a != None and a != "":
                    return True
    return False

In [ ]:
#updating the required age and content warning columns based on the rating dict
for index, row in df2.iterrows():
    if row["required_age"] == 0:
        df2.loc[index, "required_age"] = average_required_age(df1.loc[index])

for index, row in df2.iterrows():
    if row["has_content_warning"] == False:
        df2.loc[index, "has_content_warning"] = get_content_warning(df1.loc[index])

In [ ]:
#create column with boolean with True for when this column is not null
df2.loc[:,"has_dlc"]  = df1.loc[:,"dlc"].notna()
df2.loc[:,"has_movies"]  = df1.loc[:,"movies"].notna()

In [ ]:
#apply literal eval
df1["achievements"] = df1["achievements"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#create new column
df2["total_achievements"] = 0

In [ ]:
#get total achievements from dictionary and update column
for index, row in df1.iterrows():
    achievements = row["achievements"]
    if isinstance(achievements, dict):
        if "total" in achievements.keys():
            total = achievements.get("total")
            df2.loc[index, "total_achievements"] = total

In [ ]:
#create column with boolean with True for when this column is not null
df2.loc[:,"has_professional_reviews"]  = df1.loc[:,"reviews"].notna()
df2.loc[:,"has_legal_notice"]  = df1.loc[:,"legal_notice"].notna()
df2.loc[:,"has_drm_notice"]  = df1.loc[:,"drm_notice"].notna()
df2.loc[:,"has_useraccount_notice"]  = df1.loc[:,"ext_user_account_notice"].notna()

In [ ]:
#create functions to get the proportion positive, negative and the total reviews
def proportion_positive(row):
    pos = row["positive"]
    neg = row["negative"]
    if isinstance(pos, int) and isinstance(neg, int):
        total = pos + neg
        if total !=0:
            result = pos/total
            return result
    return 0

def proportion_negative(row):
    pos = row["positive"]
    neg = row["negative"]
    if isinstance(pos, int) and isinstance(neg, int):
        total = pos + neg
        if total !=0:
            result = neg/total
            return result
    return 0

def total_reviews(row):
    pos = row["positive"]
    neg = row["negative"]
    if isinstance(pos, int) and isinstance(neg, int):
        total = pos + neg
        return total
    return 0

In [ ]:
#create new columns 
df2["reviews_proportion_positive"] = 0
df2["reviews_proportion_negative"] = 0
df2["total_user_reviews"] = 0

In [ ]:
#apply functions to get values for these columns
df2["reviews_proportion_positive"] = df1.apply(proportion_positive, axis=1)
df2["reviews_proportion_negative"] = df1.apply(proportion_negative, axis=1)
df2["total_user_reviews"] = df1.apply(total_reviews, axis=1)

In [ ]:
#function to change the format for the owners range string
def format_range(value):
    parts = value.split(' .. ')
    parts = [part.strip() for part in parts]
    if len(parts) == 2:
        return f'{parts[0]}-{parts[1]}'
    else:
        return value

In [ ]:
#apply the new formatting to the df
df2["owners"] = df1["owners"].apply(format_range)

In [ ]:
#functions to change the playtime in minutes to hours and the price in cents to dollars
def playtime_hours(value):
    if isinstance(value, int):
        return value/60
    return value

def price_dollars(value):
    if isinstance(value, float):
        return value/100
    return value

In [ ]:
#apply functions and crate new columns
df2["average_playtime"] = df1["average_forever"].apply(playtime_hours)
df2["median_playtime"] = df1["median_forever"].apply(playtime_hours)
df2["initial_price"] = df1["initialprice"].apply(price_dollars)

In [ ]:
#apply literal eval
df1["tags"] = df1["tags"].apply(lambda x: handle_literal_eval(x))

In [ ]:
#get unique tags 
tags = set()
for index, row in df1.iterrows():
    tags_dict = row["tags"]
    if isinstance(tags_dict, dict):
        for key in tags_dict.keys():
            tags.add(key)

In [ ]:
#create columns for each tag
for tag in tags:
    df2[f"tag_{tag}"] = False

In [ ]:
#update the columns depending on whther that tag os stored in the dict for that row
for index, row in df1.iterrows():
    tags_dict = row["tags"]
    if isinstance(tags_dict, dict):
        for key in tags_dict.keys():
            df2.loc[index, f"tag_{key}"] = True

In [ ]:
#funtion to get the proportion of recommendations
def proportion_recommended(row):
    rec = row["total_recommendations"]
    total = row["total_user_reviews"]
    if isinstance(rec, int) and isinstance(total, int):
        if total !=0:
            result = rec/total
            return result
    return 0

In [ ]:
#create column and apply function
df2["proportion_recommended"] = df1.apply(proportion_recommended, axis=1)

Part 2 Feature Reduction and Missing Values

In [ ]:
#calculate a threshold that for 5% of data
threshold = len(df2) * 0.05
print(threshold)

In [ ]:
boolean_columns = df2.columns[df2.dtypes == bool]
drop_columns = [col for col in boolean_columns if df2[col].sum() <  threshold]
print(len(drop_columns))

In [ ]:
#get null counts
null_counts = df2.isnull().sum()
columns_null = null_counts[null_counts > 0]
columns_null

In [ ]:
#function to get the highest value out of low medium high using the index for their position in the list
def get_highest_value(row):
    cols = ['Processor', 'Storage', 'Graphics', 'Memory']
    values = []
    for col in cols:
        if pd.notnull(row[col]):
            values.append(row[col])
    if values:
        highest_value = max(values, key=lambda x: ['Low', 'Medium', 'High'].index(x))
        return highest_value
    return None

In [ ]:
#create new general system requirements colums applying the function to get the value for the hight requirement
df2['system_reqs'] = df2.apply(get_highest_value, axis=1)

In [ ]:
#make unknown category for null system reqs
df2.loc[df2["system_reqs"].isna(), 'system_reqs'] = "Unknown"

In [ ]:
#drop the specific req columns
cols_to_drop = ['Processor', 'Storage', 'Graphics', 'Memory']
df2.drop(columns=cols_to_drop, inplace=True)

In [ ]:
#for rows where either developers or publishers is missing replace it with the value in the other
df2['developers'] = df2['developers'].fillna(df2['publishers'])
df2['publishers'] = df2['publishers'].fillna(df2['developers'])

In [ ]:
#drop the ones that still have missing values
df2.dropna(subset=['developers'], inplace=True)
df2.dropna(subset=['publishers'], inplace=True)

In [ ]:
#drop games with missing release date
df2.dropna(subset=['release_date'], inplace=True)

In [ ]:
#make initial price 0 if the game is free and has a missing intial price
df2.loc[(df2["initial_price"].isna()) & (df2["is_free"] == True), "initial_price"] = 0

In [ ]:
#drop the ones that still had intial price
df2.dropna(subset=['initial_price'], inplace=True)

In [ ]:
#handling age outliers by assigning mac not extreme age of 21 when the game has a content warning and 0 otherwise
df2.loc[(df2["required_age"] > 21) & (df2["has_content_warning"] ==True), "required_age"] = 21
df2.loc[(df2["required_age"] > 21), "required_age"] = 0

Part 3 - Prepare data for use in model

In [ ]:
#tranform datatypes
df2["steam_appid"] = df2["steam_appid"].astype(int)
df2["required_age"] = df2["required_age"].astype(int)
df2["publishers"] = df2['publishers'].astype('category')
df2["developers"] = df2['developers'].astype('category')
df2["system_reqs"] = df2['system_reqs'].astype('category')
df2["owners"] = df2['owners'].astype('category')

In [ ]:
#split date data
df2['release_month'] = df2['release_date'].dt.month
df2['release_year'] = df2['release_date'].dt.year
df2['release_day_of_week'] = df2['release_date'].dt.dayofweek
df2['release_day_of_month'] = df2['release_date'].dt.day

In [ ]:
#count values in classes of owners
class_counts = df2['owners'].value_counts()
class_counts

In [ ]:
#merge the three highest categories into one
df2.loc[(df2["owners"] == "50,000,000-100,000,000")|
 (df2["owners"] == "200,000,000-500,000,000")|
  (df2["owners"] == "100,000,000-200,000,000"), "owners"] = "50,000,000+"

In [ ]:
#dict to map system req cats
system_reqs_map = {
    3: 'High', 1: 'Low', 2: 'Medium', 0: 'Unknown'
}

In [ ]:
#dict to map owner cats
owners_map = {
 1: '0-20,000',
 7: '1,000,000-2,000,000',
 10: '10,000,000-20,000,000',
 4: '100,000-200,000',
 8: '2,000,000-5,000,000',
 11: '20,000,000-50,000,000',
 2: '20,000-50,000',
 5: '200,000-500,000',
 9: '5,000,000-10,000,000',
 12: '50,000,000+',
 3: '50,000-100,000',
 6: '500,000-1,000,000'
}

In [ ]:
release_day_of_week_map = {
 5: 'Friday',
 1: 'Monday',
 6: 'Saturday',
 7: 'Sunday',
 4: 'Thursday',
 2: 'Tuesday',
 3: 'Wednesday'
}

In [ ]:
#create dict with category and catehgory code
publishers_map = {
    code: category
    for code, category in enumerate(df2['publishers'].astype('category').cat.categories)
}

In [ ]:
#create dict with category and catehgory code
developers_map = {
    code: category
    for code, category in enumerate(df2['developers'].astype('category').cat.categories)
}

In [ ]:
#funtion to reverse the key values in the dict
def reverse_dict(dictionary):
    dictionary = {v: k for k, v in dictionary.items()}
    return dictionary

In [ ]:
#apply reverse dict function
release_day_of_week_map = reverse_dict(release_day_of_week_map)
owners_map = reverse_dict(owners_map)
system_reqs_map = reverse_dict(system_reqs_map)
developers_map = reverse_dict(developers_map)
publishers_map = reverse_dict(publishers_map)

In [ ]:
#create colums for the cat codes using dictionary mapping
df1['release_day_of_week_code'] = df1['release_day_of_week'].map(release_day_of_week_map)
df1["owners_code"] = df1['owners'].map(owners_map)
df1["system_reqs_code"] = df1['system_reqs'].map(system_reqs_map)
df1["developers_code"] = df1['developers'].map(developers_map)
df1["publishers_code"] = df1['publishers'].map(publishers_map)

In [ ]:
#bins and labels for making proprotion of postive reviews categories
bins = [-0.001, 0.2, 0.4, 0.6, 0.8, 1.001]
labels = [1, 2, 3, 4, 5]

In [ ]:
#creating the categories with codes for the review proportion positive
df2["reviews_proportion_positive_bin_code"] = pd.cut(df1['reviews_proportion_positive'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
#export to csv
df2.to_csv("clean_data_with_codes.csv", index = 'False')